In [26]:
import pandas as pd
import numpy as np
import nltk
import scipy
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv("../DATASET.csv.gz", compression='gzip')

In [3]:
df['text'] = df['text'].map(lambda x: " ".join(x.split()))

In [5]:
with open("texts.txt", "w") as f:
    for x in df['text'].tolist():
        f.write(x + "\n")

In [6]:
!./fastText-0.2.0/fasttext cbow -input texts.txt -output cbow_model/model -wordNgrams 2 -epoch 25

Read 45M words
Number of words:  188467
Number of labels: 0
Progress: 100.0% words/sec/thread:   34612 lr:  0.000000 loss:  0.752368 ETA:   0h 0m


In [10]:
def normalize(arr):
    return np.array(arr) / np.linalg.norm(arr)

def read_word_emb(path):    
    emb = {}
    with open(path, "r") as m:
        n, dim = tuple(map(int, m.readline().split()))
        for l in m:
            line = l.split()
            emb[line[0]] = normalize([float(x) for x in line[1:]])
    assert n == len(emb)
    return emb

emb = read_word_emb("cbow_model/model.vec")

In [11]:
def sent_emb(text):
    """Returns mean embedding vector of all word embedding vectors in text"""
    res = []
    for w in text.split():
        if w in emb:
            res.append(emb[w])
    return np.mean(res, axis=0)

In [24]:
def split_sent(text, min_len=3, max_len=500):
    res = []
    for sent in nltk.sent_tokenize(text):
        sent = sent.split()
        if len(sent) < max_len and len(sent) > min_len:
            res.append(" ".join(sent))
    return res

res = []
for i, row in df.iterrows():
    sents = split_sent(row['text'])
    if not sents:
        print(row['text'])
    else:
        for sent in sents:
            res.append({
              "labels": row["labels"], 
              "name": row["name"],
              "number": row["number"],
              "source": row["source"],
              "sent": sent
            })
s = pd.DataFrame(res)

1. Фонды
1. Общие положения
1. Основные положения
1. Основные положения


In [25]:
cc = s[s['source'] == 'criminal_code']
co = s[s['source'] == 'criminal_court_orders']

In [ ]:
cc['vector'] = cc['sent'].map(sent_emb)
co['vector'] = co['sent'].map(sent_emb)

/home/gzhigalov/miniconda3/envs/testing/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gzhigalov/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/gzhigalov/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [38]:
co_train, co_test = train_test_split(co, test_size=0.3)

In [41]:
cc_space = scipy.spatial.KDTree(cc['vector'].tolist())

In [50]:
co = (
    co.groupby("name")
    .agg({
        "labels": "first", 
        "source": "first", 
        "sent": list, 
        "vector": list})
    .reset_index(drop=True)
)

In [58]:
cc.iloc[[1,2,3]]['number'].tolist()

[1.0, 1.0, 2.0]

In [67]:
def find_nearest_by_sent(vectors, k=5):
    res = []
    for vec in vectors:
        dist, idx = cc_space.query(vec, k=k)
        articles = cc.iloc[idx]["number"].tolist()
        res.append(list(zip(dist, articles)))
    return res

In [ ]:
co["labels"] = co["labels"].map(lambda x: list(map(float, x.split())))

In [ ]:
co.to_csv('co.csv.gz', compression='gzip', index=False)
cc.to_csv("cc.csv.gz", compression='gzip', index=False)